In [1]:
# feedparser is a python library to parse python feeds
import feedparser

import pandas as pd


In [2]:
# connecting to mongo Database

from pymongo import MongoClient


# key credential for mongo cluster
MONGO_CLIENT_KEY = "you wish"

# using MongoClient  Credentials & instance   to connect to 
client = MongoClient(MONGO_CLIENT_KEY)
db = client.finalAssignment

In [3]:
# reading data from collectioon and getting words using textblob
from textblob import TextBlob as tb

# cleaning data removing tags
def cleanData(str):
    filterWords = list()
    blob = tb(str)
    filterTags = list(['VBD', 'DT','TO', 'CD', 'CC', 'IN', 'VBZ', 'VB'])
    for word,tag in blob.tags:
        if (tag not in filterTags): 
            filterWords.append(word.lemmatize())
    return " ".join(filterWords)

In [4]:
# for anlysing data
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def getVaderPoolarityScores(string):
    return analyser.polarity_scores(string)

            
# get string sentiment
def getSentiment(string):
    polarityScores = getVaderPoolarityScores(string)
    result = ''
    
    if polarityScores['compound'] < -0.5 and polarityScores['neg']-polarityScores['pos'] > 0 :
        result = 'Negative'
    elif polarityScores['compound'] > 0.5 and polarityScores['pos']-polarityScores['neg'] > 0 :
        result =  'Positive'
    else:
        result = 'Neutral'
    
    polarityScores['sentiment'] = result
    return polarityScores


In [5]:
def addToDB(dbInstanceCol, item):
    dbInstanceCol.insert_one(item)

# parse item data for each entries
def parseEntries(item):
        title = item.title
        image = item.image.href
        summary = item.summary
        
        cleanSummary = cleanData(summary)
        summarySentimetnt = getSentiment(summary)

        return {
            'titile': title,
            'summary': summary,
            'cleanSummary': cleanSummary,
            'image': image,
            'vader_compound': summarySentimetnt['compound'],
            'vader_neg': summarySentimetnt['neg'],
            'vader_pos': summarySentimetnt['pos'],
            'sentiment': summarySentimetnt['sentiment']
            
        }

In [6]:
def  parseRSS(source, collection):
    s = feedparser.parse(source)
    entries = s.entries
    for item in entries[:100]:
        dbObj = parseEntries(item)
        addToDB(collection, dbObj)
    

In [7]:
source1 = "http://joeroganexp.joerogan.libsynpro.com/rss"
hashtag1Col = db.source1

parseRSS(source1, hashtag1Col)

# source1df.columns = ['titile', 'summary', 'cleanSummary', 'image', 'sentiment'] 

# source1df.to_excel("source1df.xlsx")



In [8]:
source2 = "https://feeds.npr.org/510312/podcast.xml"
hashtag2Col = db.source2
parseRSS(source2, hashtag2Col)

In [9]:
source3 = "https://www.omnycontent.com/d/playlist/aaea4e69-af51-495e-afc9-a9760146922b/44bbf446-4627-4f83-a7fd-ab070007db11/72b96aa8-88bd-480a-87af-ab070007db36/podcast.rss"
hashtag3Col = db.source3
parseRSS(source3, hashtag3Col)

In [10]:
source4 = "https://mcsorleys.barstoolsports.com/feed/call-her-daddy"
hashtag4Col = db.source4
parseRSS(source4, hashtag4Col)

In [11]:
source5 = "https://podcastfeeds.nbcnews.com/dateline-nbc"
hashtag5Col = db.source5
parseRSS(source5, hashtag5Col)